q-1-3.py

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree, preprocessing
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
eps = np.finfo(float).eps

Reading data from csv

In [2]:
df = pd.read_csv('train.csv')

spliting features and class label and dropping numerical data.

In [3]:
Y = df.left
X = df.drop(['left'],axis='columns')

doing One hot Encoding to convert multi-valued features to binary.

In [4]:
# axis = 0 means row and axis= 1 means columns. 
# default is 0. 
Z = pd.concat([X,pd.get_dummies(X['sales'], prefix='sales')],axis='columns')
Z = pd.concat([Z,pd.get_dummies(Z['salary'], prefix='salary')],axis='columns')
Z = Z.drop(['sales','salary'],axis='columns')

Split train and test data 

In [5]:
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.2)
X_train, X_test, Y_train, Y_test = train_test_split(Z, Y,test_size=0.2)
df_ent = pd.concat([X_train,Y_train],axis=1)
df_gini = pd.concat([X_train,Y_train],axis=1)
df_mis_rate = pd.concat([X_train,Y_train],axis=1)

In [6]:
def mymodel(df1, flag ):
    def get_impurity( df , flag ):
        Class = df.keys()[-1]
        if flag == 1:
            entropy = 0
            values = df[Class].unique()
            for value in values:
                fraction = float(df[Class].value_counts()[value])/len(df[Class])
                entropy += -fraction*np.log2(fraction+eps)
            return entropy
        if flag == 2:
            entropy = 1
            values = df[Class].unique()
            for value in values:
                fraction = float(df[Class].value_counts()[value])/len(df[Class])
                entropy *= fraction
            return 2*entropy
        if flag == 3:
            entropy = 1
            values = df[Class].unique()
            for value in values:
                fraction = float(df[Class].value_counts()[value])/len(df[Class])
                entropy = min(fraction , 1-fraction)
            return entropy

    def get_impurity_attr( df,  attribute, flag ):
        if flag == 1:
            Class = df.keys()[-1]
            target_variables = df[Class].unique()  
            variables = df[attribute].unique()
            entropy2 = 0
            for variable in variables:
                entropy = 0
                for target_variable in target_variables:
                    num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
                    den = len(df[attribute][df[attribute]==variable])
                    fraction = num/(den+eps)
                    entropy += -fraction*np.log2(fraction+eps)
                fraction2 = float(den)/len(df)
                entropy2 += -fraction2*entropy
            return abs(entropy2)
        if flag == 2:
            Class = df.keys()[-1]
            target_variables = df[Class].unique()  
            variables = df[attribute].unique()
            entropy2 = 0
            for variable in variables:
                entropy = 1
                for target_variable in target_variables:
                    num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
                    den = len(df[attribute][df[attribute]==variable])
                    fraction = num/(den+eps)
                    entropy *= fraction
                entropy *= 2
                fraction2 = float(den)/len(df)
                entropy2 += fraction2*entropy
            return entropy2
        if flag == 3:
            Class = df.keys()[-1]
            target_variables = df[Class].unique()  
            variables = df[attribute].unique()
            entropy2 = 0
            for variable in variables:
                entropy = 1
                for target_variable in target_variables:
                    num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
                    den = len(df[attribute][df[attribute]==variable])
                    fraction = num/(den+eps)
                    entropy = min(fraction, 1 - fraction)
                fraction2 = float(den)/len(df)
                entropy2 += fraction2*entropy
            return entropy2

    def attr_to_select(df,flag):
        Entropy_att = []
        IG = []
        for key in df.keys()[:-1]:
            IG.append(get_impurity(df, flag)-get_impurity_attr(df,key, flag))
        return df.keys()[:-1][np.argmax(IG)]

    def get_subtable(df, node, value):
        return df[df[node] == value].reset_index(drop=True)

    class_ent = get_impurity(df1, flag)
    def find_IG( df, val, attr , flag ):
        left = df[df[attr] < val ].reset_index(drop=True)
        right = df[df[attr] >= val ].reset_index(drop=True)
        left_imp = get_impurity(left, flag)
        right_imp = get_impurity(right, flag)
        return class_ent - ((float(len(left))/(len(df)+eps) * left_imp)+( float(len(right))/(len(df)+eps) * right_imp))

    def split_numerical( attr , Y , attr_name , flag):
        max_ig = 0
        max_split = None
        pair = pd.concat([attr, Y], axis='columns')
        pair = pair.sort_values(by =attr_name).reset_index()
        for i in xrange( len(attr)-1):
            if pair['left'][i] != pair['left'][i+1]:
                cur_ig = find_IG( pair, float(pair[attr_name][i] + pair[attr_name][i+1])/2 , attr_name, flag )
                if cur_ig > max_ig:
                    max_ig = cur_ig
                    max_split =  float(pair[attr_name][i] + pair[attr_name][i+1])/2
        return max_split

    def change_actual( df, val, attr ):
        df.loc[df[attr] < val, attr ] = 0
        df.loc[df[attr] >= val, attr ] = 1
        return

    num_attr = [ 'number_project','last_evaluation', 'satisfaction_level','average_montly_hours','time_spend_company']
    split_dict = {}
    for attr in num_attr:
        split_val = split_numerical(df1[attr], df1['left'],attr, flag)
        split_dict[attr] = split_val
    print split_dict

    def preprocess(df,split_dict):
        for key,value in split_dict.iteritems():
            change_actual(df, value, key)

    preprocess(df1,split_dict)

    class Node:
        def __init__(self, feature):
            self.feature = feature
            self.positive = 0
            self.negative = 0
            self.left = None
            self.right = None

    def build_Tree(df, flag):
        if len(df.columns) == 1:
            return None
        node_to_split = attr_to_select(df,flag)

        root = Node(node_to_split)
        root.positive = len( df[df['left']==1]['left'] )
        root.negative = len( df[df['left']==0]['left'] )

        subtable0 = get_subtable(df,node_to_split,0)
        subtable1 = get_subtable(df,node_to_split,1)

        subtable0 = subtable0.drop(node_to_split,axis=1)
        subtable1 = subtable1.drop(node_to_split,axis=1)

        clValue0,counts0 = np.unique(subtable0['left'],return_counts=True)
        clValue1,counts1 = np.unique(subtable1['left'],return_counts=True)

        if len(counts0)>1:
            root.left = build_Tree(subtable0,flag)
        if len(counts1)>1:
            root.right = build_Tree(subtable1,flag)

        return root

    def rec_predict(df,root,Y1):
        if root == None:
            return None

        if root.right==None and root.left==None:
            Y1.append(1 if root.positive > root.negative else 0)
            return

        if root.right==None and df[root.feature] == 1:
            Y1.append(1 if root.positive > root.negative else 0)
            return 
        if root.left == None and df[root.feature] == 0:
            Y1.append(1 if root.positive > root.negative else 0)
            return

        if df[root.feature]==0:
            rec_predict(df,root.left,Y1)
        else:
            rec_predict(df,root.right,Y1)

    def predict(df,root,Y1):
        for col,row in df.iterrows():
            rec_predict(row,root,Y1)

    root = build_Tree(df1,flag)
    pd.options.mode.chained_assignment = None
    X_test_copy = X_test.copy(deep=True)
    Yp=[]
    preprocess(X_test_copy,split_dict)
    predict(X_test_copy,root,Yp)

    print confusion_matrix(Y_test,Yp)
    print classification_report(Y_test,Yp)
    print accuracy_score(Y_test, Yp)

In [7]:
mymodel(df_ent, 1)

{'satisfaction_level': 0.46499999999999997, 'last_evaluation': 0.58, 'average_montly_hours': 287.5, 'time_spend_company': 3.0, 'number_project': 2.5}
[[1642   61]
 [ 241  304]]
              precision    recall  f1-score   support

           0       0.87      0.96      0.92      1703
           1       0.83      0.56      0.67       545

   micro avg       0.87      0.87      0.87      2248
   macro avg       0.85      0.76      0.79      2248
weighted avg       0.86      0.87      0.86      2248

0.8656583629893239


In [8]:
mymodel(df_gini, 2)

{'satisfaction_level': 0.46499999999999997, 'last_evaluation': 0.58, 'average_montly_hours': 275.0, 'time_spend_company': 3.0, 'number_project': 2.5}
[[1664   39]
 [ 215  330]]
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      1703
           1       0.89      0.61      0.72       545

   micro avg       0.89      0.89      0.89      2248
   macro avg       0.89      0.79      0.83      2248
weighted avg       0.89      0.89      0.88      2248

0.8870106761565836


In [9]:
mymodel(df_mis_rate, 3)

{'satisfaction_level': 0.11499999999999999, 'last_evaluation': 1.0, 'average_montly_hours': 275.0, 'time_spend_company': 4.0, 'number_project': 2.5}
[[1625   78]
 [ 280  265]]
              precision    recall  f1-score   support

           0       0.85      0.95      0.90      1703
           1       0.77      0.49      0.60       545

   micro avg       0.84      0.84      0.84      2248
   macro avg       0.81      0.72      0.75      2248
weighted avg       0.83      0.84      0.83      2248

0.8407473309608541
